# Air Sac Tracking Prototype code
Lara S. Burchardt & Wim Pouw

lara.burchardt@donders.ru.nl

Next to body movements and acoustics, we would also like to track the air sac's inflation of the Siamangs. The air sac naturally forms a spherical(3D) or circular (2D) shape, and such shapes are retrievable from an image using the hough transform, and a bit of pre-processing of the images to get the optimal representation of the relevent edges of the air sac.

This code takes as input a sample video with a close up of a Siamang, and then tracks the air sac when it takes a sufficiently circular shape. The result is shown below; it is not perfect, but with a bit of smoothing this can function as a good air sac tracker. This code is very much under development, there are many ways to improve further.

In [2]:
# import the necessary packages
import numpy as np
import argparse
import cv2
import pandas as pd
from skimage import io, feature, color, measure, draw, img_as_float
import numpy as np
import csv
import random2
import statistics
import scipy
from scipy import signal
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import itertools
import os
from os import listdir
from os.path import isfile, join


#resused code from 
#https://pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
#https://stackoverflow.com/questions/31705355/how-to-detect-circlular-region-in-images-and-centre-it-with-python

## Define Prepocessing Function

In [3]:
#define parameters for HoughTransform outside of function to be able to save and manipulate easier
def hougdraw(submitted_image, dp = 1, mindist = 10000, param1 = 10, param2=22, minradius = 5, maxradius=250):
    circles = cv2.HoughCircles(submitted_image, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist = mindist,  
                               param1 = param1, param2 = param2, 
                               minRadius = minradius, maxRadius = maxradius)
    if circles is not None:
        circles = np.round(circles[0, 0:1]).astype("int")
        circle1 = circles[0,0]
        circle2 = circles[0,1]
        circle3 = circles[0,2]
        for(x, y, r) in circles:
            cv2.circle(submitted_image, (x, y), r, (255, 255, 0), 2) #version without drawing roi back on whole image 
    return(submitted_image)
    
# define function for preprocessing
#def preprocessing(image, medianblur = 27, 
#                              dilation = 7, 
#                              alpha = 2, 
#                              beta= 30,
#                              thresh_div_1=10,
#                              thresh_div_2=15):
#    #image0 = hougdraw(image)
#    #convert to grayscale
#    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#    #brightness change
#    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
#    #set dynamic tresholds for canny (and thus also for hough)
#    mean_intensity = np.median(gray)
#    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/thresh_div_1))
#    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/thresh_div_2))    
#    image1_h = hougdraw(gray,param1= threshold1, param2 =threshold2)
#    #blur
#    image2 = cv2.medianBlur(gray, medianblur)
#    image2_h = hougdraw(image2.copy(),param1= threshold1, param2 =threshold2)
#    #dynamic thresholds for canny edge detection based on intensity of image
#    #Thresholds one standard deviation above and below median intensity
#    #edge detection
#    image3 = cv2.Canny(image2, threshold1, threshold2)
#    image3_h = hougdraw(image3.copy(), param1= threshold1, param2 =threshold2)
#    #dilation and second blur
#    submitted = cv2.dilate(image3, None, iterations= dilation)  
#    image4 = cv2.medianBlur(submitted, medianblur) 
#    image4_h = hougdraw(image4.copy(), param1= threshold1, param2 =threshold2)
#    #add hough
#    return image1_h, image2_h, image3_h, image4_h
############################################

def preprocessing(image, medianblur = 27, 
                              dilation = 7, 
                              alpha = 2, 
                              beta= 30,
                              thresh_div_1=10,
                              thresh_div_2=15):
    #image0 = hougdraw(image)
    #convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    #brightness change
    gray = cv2.convertScaleAbs(gray, alpha = alpha, beta = beta)
    #set dynamic tresholds for canny (and thus also for hough)
    mean_intensity = np.median(gray)
    threshold1 = int(max(0, (1.0 - 0.33) * mean_intensity/thresh_div_1))
    threshold2 = int(min(255, (1.0 + 0.33) * mean_intensity/thresh_div_2))    
    #blur
    image2 = cv2.medianBlur(gray, medianblur)
    #dynamic thresholds for canny edge detection based on intensity of image
    #Thresholds one standard deviation above and below median intensity
    #edge detection
    image3 = cv2.Canny(image2, threshold1, threshold2)
    #dilation and second blur
    submitted = cv2.dilate(image3, None, iterations= dilation)  
    image4 = cv2.medianBlur(submitted, medianblur) 
    #add hough
    image4 = np.float32(image4)
    return image4, threshold1, threshold2

def matriximages(outputname, wimagenumber, himagenumber, imagelist, marginbetweenimages, vertnames, hornames):
    """
    matrix images takes in the full path name of the outputfolder for storing the images
    it takes as input a list of full path names of the images
    the height is given in image numbers and same for width
    the vert names is a list of column names
    the hor names is a list of row names
    """
    margin = marginbetweenimages
    #img = imagelist  
    w = wimagenumber
    h = himagenumber
    n = w*h

    #imgs = [cv2.imread(i) for i in imagelist]
    imgs = imagelist
    #if any(i.shape != imgs[0].shape for i in imgs[1:]):
    #    raise ValueError('Not all images have the same shape.')

    img_h, img_w = imgs[0].shape#img_h, img_w, img_c = imgs[0].shape

    m_x = 0
    m_y = 0
    if marginbetweenimages is not None:
        margin = marginbetweenimages
        m_x = int(margin*img_w)
        m_y = int(margin*img_h)
        
    imgmatrix = np.zeros((img_h * h + m_y * (h - 1),
                          img_w * w + m_x * (w - 1)),
                         np.uint8)

    imgmatrix.fill(255)    

    positions = itertools.product(range(w), range(h))
    for (x_i, y_i), img in zip(positions, imgs):
        x = x_i * (img_w + m_x)
        y = y_i * (img_h + m_y)
        imgmatrix[y:y+img_h, x:x+img_w] = img #imgmatrix[y:y+img_h, x:x+img_w, :] = img
    #add text
    font = cv2.FONT_HERSHEY_DUPLEX
    cv2.putText(imgmatrix, hornames[0], (0*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[1], (1*img_w,  int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[2], (2*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[3], (3*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, hornames[4], (4*img_w, int(img_h/4)), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[0], (0, 1*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[1], (0, 2*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[2], (0, 3*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    cv2.putText(imgmatrix, vertnames[3], (0, 4*img_h), font, 10, (255, 255, 255), 5, cv2.LINE_AA)
    #cv2.putText(imgmatrix, vertnames[4], (0, 5*img_h), font, 10, (0, 255, 0), 5, cv2.LINE_AA)
    cv2.imwrite(outputname, imgmatrix)   
    
    print('done, look in your folder: '+ outputname)
    
    
def roidefinition(video, height_buffer, width_buffer, dp, minRadius, maxRadius):
    cap = cv2.VideoCapture(video)
    totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    #frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    #frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    #fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second
    #set up empty output dataframe
    column_names = ['x','y', 'r', 'frame'] # we are going to collect hough generated x,y,r,frame from random set of frames
    df_round_1 = pd.DataFrame(columns = column_names) 
    vector = range(0, int(totalFrames), 1)
    samp = random2.sample(vector, 45) #50 is the setting
    for rand in samp:
        # set frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES,rand)
        ret, frame = cap.read()
        ############################detect circles   
        output=frame.copy()
        # transform to grayscale image only using the roi part of the image
        image4, param1, param2 = preprocessing(image=output)
        final_im = cv2.normalize(src=image4, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        #final_im = image4
        circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                                   dp = dp,minDist = minDist,  
                                   param1 = param1,param2 = param2, 
                                   minRadius = minRadius, maxRadius = maxRadius)
        #if circles is not None:
        #    circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            #circles = np.round(circles[0, 0:1]).astype("int")
            if circles is not None:
                circles = np.round(circles[0, 0:1]).astype("int")
                circle1 = circles[0,0]
                circle2 = circles[0,1]
                circle3 = circles[0,2]
            #save it to a row
            if circles is None:
                circle1 = "NA"
                circle2 = "NA"
                circle3 = "NA"
        new_row = [circles[0,0], circles[0,1],circles [0,2], rand]
        df_round_1.loc[len(df_round_1)] = new_row

    #when collected take some info
    median_x = df_round_1['x'].median() #order in pos : same as in original dataframe, so x,y, r
    median_y = df_round_1['y'].median()
    max_r = df_round_1['r'].max()
    min_r = df_round_1['r'].min()
    # order of parameters saved as roi with cv2.selectROI [Top_Left_X, Top_Left_Y, Width, Height]

    # explanation: height_1 is the maximum radius detected plus 30 pixels, that is used to determine the position of the 
    # upper left corner of the roi, same goes for width_1, both are currently the same, but that could change, so they are both
    # coded independently
    height_1 = max_r + height_buffer #i.e.100, in pixels
    width_1 = max_r + width_buffer #250, in pixels
    pos_x = median_x - width_1
    pos_y = median_y - height_1  
    # to be consistent with the roi nomenclature, we then calculate the width/height of the whole roi which is width_1 *2
        # all those values will then be used to crop picture in next round of tracking
    width_2 = width_1 * 2
    height_2 = height_1 *2
    #
    roi = [pos_x, pos_y, width_2, height_2]
    cap.release() #release the video
    return roi

# track a video with template matching

In [31]:
# preprocessing, choosing a roi on the tracking results of 50 random samples from input file
#https://github.com/patchy631/machine-learning/blob/main/computer_vision/cv2_edge_detection.ipynb
import sys
from skimage.measure import compare_ssim
from skimage.transform import resize

############settings that worked: c1_5_c2_10_al_1_b_12_dil_7_blur_27
alpha = 2
beta = 20
dp = 1
dilation = 5
phase1_medianblur = 27
#cannyt1 = 5
#cannyt2 = 12 
minDist = 10000
minRadius = 5 #minus 2times std
maxRadius = 260 #dynamic? 
###################
videofolder= '../Video/'
nameforfiles = 'example1'
videofilename = nameforfiles +'.mp4'
# Opens the Video file
cap = cv2.VideoCapture(videofolder+videofilename)
totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
frameWidth = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
frameHeight = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = cap.get(cv2.CAP_PROP_FPS)   #fps = frames per second

#set up empty output dataframe
column_names = ['x','y', 'r', 'frame'] # we are going to collect hough generated x,y,r,frame from random set of frames
df_round_1 = pd.DataFrame(columns = column_names) 
vector = range(0, int(totalFrames), 1)
samp = random2.sample(vector, 40) #50 is the setting

out = cv2.VideoWriter('./Output/videos/output_'+videofilename,cv2.VideoWriter_fourcc(*'MP4V'), fps, 
                      (int(frameWidth), int(frameHeight)))
t_image = cv2.imread('./Output/videos/template_test.jpg')
t_gray_image = cv2.cvtColor(t_image, cv2.COLOR_BGR2GRAY)

#main loop                      
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    # set frame position

    ############################detect circles   
    output=frame.copy()
    # transform to grayscale image only using the roi part of the image
    image4, param1, param2 = preprocessing(image=output)
    final_im = cv2.normalize(src=image4, dst=0.01, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    circles = cv2.HoughCircles(final_im, cv2.HOUGH_GRADIENT, 
                               dp = dp,minDist =10,#when you lower mindist, more candidate circles are given  
                               param1 = param1,param2 = param2, 
                               minRadius = minRadius, maxRadius = maxRadius)
    print(len(circles[0])) #how many circles?
    #######NEW CHECK WIM to check for multiple circles and choose best match based on template matching
    # Iterate over the detected circles
    fits = []
    circles = circles[0]
    print(len(circles))
    for i in circles:
        # Get the coordinates of the center and the radius of the circle
        x, y, r = i[0], i[1], abs(i[2])
        tim = frame.copy()
        tim = cv2.cvtColor(tim, cv2.COLOR_BGR2GRAY)
        template1 = tim[int(y-r):int(y+r), int(x-r):int(x+r)]

        height1, width1 =  template1.shape
        height2, width2 =  t_gray_image.shape
        res = 0
        # Check if the images are the same size; and than compute a distance measure
        if (height1 < height2) & (width1 < width2) & (height1 != 0) & (width1 !=0): 
            # get two images - resize both to 1024 x 1024
            img_a = resize(template1, (2**10, 2**10))
            img_b = resize(t_gray_image, (2**10, 2**10))

            # measure of the structural similarity between the images
            res, diff = compare_ssim(img_a, img_b, full=True)
            print(res)
        #    res = cv2.matchTemplate(t_gray_image, template1, cv2.TM_CCOEFF_NORMED)   
        #    np.max(maxs)            
        fits.append(res)
    #which fits has the highest match? 
    index = fits.index(max(fits))
    print(index)
    circles = circles[index]   
    #if circles is not None:
    #    circles = np.round(circles[0, 0:1]).astype("int")
    if circles is not None:
        #circles = np.round(circles[0, 0:1]).astype("int")
        if circles is not None:
            circles = np.round(circles).astype("int")
            circle1 = circles[0]
            circle2 = circles[1]
            circle3 = circles[2]
            cv2.circle(frame, (int(circle1), int(circle2)), circle3, (200, 0, 0), 2)
        #save it to a row
        if circles is None:
            circle1 = "NA"
            circle2 = "NA"
            circle3 = "NA"
    out.write(frame)
    #new_row = [circle1, circle2,circle3]
    #df_round_1.loc[len(df_round_1)] = new_row
    cv2.waitKey(1)
    
# cleaning up
out.release()
cap.release()
cv2.destroyAllWindows()


1105
1105


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: UserWarning: DEPRECATED: skimage.measure.compare_ssim has been moved to skimage.metrics.structural_similarity. It will be removed from skimage.measure in version 0.18.


0.827359638002021
0.7450113857568768
0.7508386682059596
0.7063180125164485
0.7403903520383979
0.7137047528575505
0.7595753913000778
0.6902202448296333
0.7141039959948631
0.8223328326651435
0.7169426038494248
0.7302576580505342
0.7981426800071595
0.6465695448391812
0.7487341337534417
0.7371470291829705
0.6609757551315985
0.6719875563214759
0.7217896622745326
0.6828386047562505
0.7326227276762932
0.7686610544490755
0.7422638423597252
0.7368690814968937
0.6832350444193845
0.6940216203841001
0.6393554911371676
0.7416688055685805
0.7546805252808823
0.7438743660044014
0.810185359923573
0.659478704765676
0.6956784333832099
0.6698616629927712
0.7445862803729623
0.7306020250023207
0.7036782564519859
0.6688659951790485
0.693770235204112
0.8441582766325669
0.7323763301943321
0.719593388939214
0.7080161765760591
0.7673073393639177
0.770135136522692
0.8252299194025466
0.6429743149198851
0.6934217662561175
0.7255992260907114
0.6757403327393073
0.7795816798375577
0.7487432908845747
0.7251426336117388

0.7438913128310157
0.7490098939153369
0.7305605937428074
0.7303556986103537
0.4119209178704519
0.7591920618003463
0.738207260745033
0.7243183327172561
0.7578025659806377
0.6738781385513396
0.7234289237736508
0.7414689036658746
0.789249808733493
0.7610074397472888
0.6720040817664432
0.7427740907149665
0.641305352440302
0.7421740017673089
0.5814290093320101
0.6911536286239555
0.6722255133805068
0.7997120850478906
0.7437333753636564
0.7230670630405709
0.7258352666584269
0.7214536916559473
0.7494260457273804
0.6405986051556828
0.7193843736479143
0.6737432093702407
0.7310842702569516
0.7370745241159478
0.679291423136037
0.735945302709711
0.7725352216820165
0.7153542309185541
0.7414651162805302
0.6376395464516589
0.772797447398271
0.7237337462833454
0.665160829808706
0.6514874982142176
0.5989591636260317
0.6612181527511477
0.7626109587107223
0.7682154673547941
0.7259810864380488
0.7117054350037373
0.7111476485123627
0.8036566001052416
0.7748326394514276
0.7332073029968222
0.7497442182755651


0.6614804821329442
0.746182201247716
0.7150905320949604
0.6770156562664922
0.7212887966927037
0.3945894945280073
0.6494051724141132
0.7867609482002484
0.6871578452441831
0.7182304418747035
0.7415899669446564
0.7457893604693461
0.7650354441686705
0.7407668181259689
0.7270049265523971
0.6385428369600883
0.6831131370585042
0.6791797281805031
0.6963927830402264
0.6669381498918243
0.7151260654875083
0.7776848657798391
0.7334489304012907
0.6834568557669369
0.6434944038387284
0.7477763939525154
0.7463316284192308
0.5898080796118249
0.6429564260016514
0.7093209773188083
0.7534152374438392
0.730235746952133
0.749714765428384
0.6447622419957149
0.584209926703492
0.6842897381045941
0.7060639003522587
0.6905490909537865
0.6767375218434286
0.75181197094615
0.7271873197713885
0.7982338445125362
0.7384735812081178
195
1042
1042
0.7479373134687451
0.7597413341439861
0.753137477865971
0.6830733801582551
0.7106821611935111
0.7600008406178828
0.7420972599556355
0.7175832623914797
0.7794891115925141
0.640

...

0.7604686333238693
0.7106329607181208
0.7160235105708331
0.7121556026858138
0.5728682427698816
0.7209153548562446
0.7400823087765841
0.7792231361159828
0.6635280891685571
0.7654150357665365
0.5823380027601733
0.7750273009700486
0.7024792371722757
0.6662010816681405
0.7495628558967864
0.8017501111057826
0.6884089662709807
0.7724612729266895
0.728093738701786
0.6804219540875731
0.7736601793779907
0.6538995143930837
0.6881174537294874
0.757150241694083
0.7710788588390517
0.667704274726844
0.7561124161684867
0.6488697356634607
0.7335355249074154
0.6782001839967596
0.7467835377095894
0.768333117484944
0.7090249453883278
0.7808960666405413
0.5824288077016779
0.6666507619135712
0.8006536696194139
0.6469773348720226
0.6574511012225204
0.6619679255315475
0.6723595925591231
0.7397938848962505
0.7132355239102899
0.7586460222064303
0.7080270899519353
0.7011917063145856
0.6508964519864924
0.5921970608679364
0.7740242957114041
0.7477671715749169
0.3874585612673652
0.7224761420864769
0.73212887924747

0.8365043420476321
0.7790469659881388
0.7167574374597413
0.746472648414859
0.6524586188051863
0.7378434143359486
0.39632968375524325
0.7057984150031821
0.7153791566519377
0.6630192847019548
0.6698533007558559
0.7795307268957663
0.6740373673811069
0.7761812722634388
0.6918052438169737
0.7668241799895172
0.61922782005909
0.718348142765724
0.6824980672910789
0.7190590408386376
0.7257122915159707
0.7683463995926654
0.6652284220867949
0.7325836591282632
0.7533128117882225
0.5861767975803333
0.8377686760904935
0.8494054740556775
0.7584702870174903
0.6949416448007333
0.7172635169444894
0.6191080113470657
0.7466527628212658
0.700421443191197
0.7035309846699385
0.720852574711539
0.7565294563566423
0.7367977976781633
0.748623950210663
0.6767167719457308
0.6603593912010947
0.7696585339951819
0.7175151836870977
0.7693416525156535
0.7789729989581234
0.8418769673285944
0.6965470857190832
0.7715138854310557
0.6884656045875271
0.7264622609086062
0.6956803470277148
0.7619475020602069
0.7182601511039052

0.7119766271424941
0.7638353763022296
0.7144941927376525
0.7148222522503204
0.6641284917673628
0.7087478114931819
0.7028373444661431
0.679487605830858
0.7617589926688113
0.8043159733247179
0.796995800089974
0.6290675502591758
0.74259806662598
0.6952585923620296
0.6844597045093349
0.7033139283734615
0.7486095407234339
0.6641620066247103
0.7110158147411513
0.7380814496367812
0.7244513533461644
0.7941999769186358
0.7887253609309643
0.664843557118049
0.7221560460238933
0.6340622817799431
0.6958424345974668
0.6251293103278438
0.7291667850596739
0.7330376660465875
0.749131239648953
0.6138227813065846
0.6362283965820731
0.7675932724896574
0.7238721509737944
0.7757718137450961
0.565952552730326
0.7573474241918041
0.7138399859546352
0.7101207427346969
0.7480271988106852
0.7064928130033011
0.7446572171283513
0.6899017254347616
0.65169505333923
0.7328032468244253
0.6626801435337477
0.49432110121258727
0.7178536085340899
0.7195720122796819
0.7284501917935606
0.37921337074278166
0.7612987274791365


0.6984970846781262
0.6812351759992311
0.5758077074708657
0.697464007037951
0.6632295832133077
0.3891616195837409
0.7155346255817535
0.7041498516153877
0.7028886659313951
0.6834675086811199
0.6948897061936917
0.4150785644761747
0.7763790171156507
0.7471056778317898
0.7962257953891682
0.390936338701136
0.3892342793560471
0.7655380770567366
0.7904782540817207
0.7229333726061802
0.694947319356084
0.6944283457016107
0.7366304438873682
0.6699883667047144
0.4211426017230205
0.6579145908277879
0.7107352271683247
0.644444628276486
0.724200544112007
0.5466511584201226
0.6866730623838301
0.7007330086893515
0.793318279356954
0.37450814316474873
0.7575020529119192
0.6626095020200633
0.6369224324197437
0.6831551543355591
0.7068329631002072
0.754228134180614
0.7810833838872115
0.7300476343905762
0.6940848018885577
0.7133073350845729
0.7141045049473332
0.7818084489229249
0.6503240970365605
0.3837789123245546


KeyboardInterrupt: 

In [32]:
out.release()

4


array([1316,  500,  114])

False

143.1

[1184.5  560.5  140.9]
[1253.5  401.5  237.4]
[1498.5  821.5  214.5]
[1543.5  577.5  212.9]
[1618.5  691.5  177.2]
[1527.5  265.5  259.4]
[1604.5  485.5  171.2]
[1435.5  928.5  184.8]
[1386.5  785.5   97.3]
[1378.5  472.5   94.5]
[1395.5  652.5  135.3]
[1546.5  968.5  165.8]
[1209.5  720.5  122.6]
[1197.5  290.5  113.4]
[1148.5  387.5  227.1]
[1610.5  175.5  216. ]
[1781.5  786.5  132.3]
[1424.5 1031.5  216.2]
[1078.5  258.5   32.3]
[1498.5  676.5  183.9]
[1688.5  580.5   96.9]
[1423.5  349.5  133.4]
[1585.5   30.5   55.1]
[191.5 712.5  73.9]
[1602.5  843.5   88.1]
[1181.5  858.5  104.8]
[1665.5  403.5   64.9]
[324.5 523.5  47.2]
[1766.5  652.5  103.6]
[ 751.5 1024.5   48.1]
[1276.5  499.5  110.6]
[714.5 367.5  51.2]
[1774.5   89.5   65.5]
[1498.5  416.5   78.3]
[ 82.5 655.5  64.6]
[1100.5  751.5  144.7]
[1350.5  272.5   46.3]
[955.5 342.5 159.3]
[1726.5  874.5  100.1]
[1507.5  121.5   16.3]
[1279.5  637.5   40.2]
[1322.5 1006.5   46.3]
[1846.5  993.5    9.6]
[1720.5  316.5   60.7]
[62

In [26]:
out.release()